### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)

In [2]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product

conds = ['low','high']
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/LDA_PAC_E_EpiPerf_LowHigh_BBG/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}_odor_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
freqs = ['0_theta', '1_alpha']#['0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
min_win = 1
th = '01'
for freq in freqs:
    # Load the npz file with su_codes, xyz of signif and not signif elec
    mat = np.load(f_form.format('All_subjects',freq))
    subjects, s_th, s_da = mat['su_codes'],mat['s_th_'+th], mat['s_da']
    #print(subjects.shape, s_th05.shape, s_th01.shape, s_da.shape)
    su_codes = np.unique(subjects)
    mask = np.array([])
    for su in su_codes:
        n_elecs = s_th[np.where(subjects ==su)].shape[0]
        thresh = np.amax(s_th[np.where(subjects==su)])
        da = s_da[np.where(subjects==su)]
        print(n_elecs,da.shape,thresh)
        for elec in range(n_elecs):
            da_elec = da[elec]
            pvals = np.array([])
            if da_elec > thresh:
                pval = 1
            if da_elec <= thresh:
                pval = 0
            pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
            #criteria for an elec to be significant
            pvals = np.ravel(pvals)
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= min_win]
            if len(signif) >=1:
                remove = False
            else:
                remove = True
            mask = np.hstack((mask, remove)) if np.size(mask) else remove
    print(freq, len(mask),len(np.where(mask == False)[0]))
    np.save(f_save.format('All_subjects',freq,min_win,th),mask)

-> Olfacto loaded


FileNotFoundError: [Errno 2] No such file or directory: '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/figure/LDA_PAC_E_EpiPerf_LowHigh_BBG/All_subjects_sources_0_theta_odor_low_high_sel_physFT.npz'

### 2- Create masks - min patients

In [11]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product
from PIL import Image
score = 'Epi'
conds = ['low','high']
th = '05'
###############################################################################
st = study('Olfacto')
path_classif = join(st.path, 'classified/1_Classif_PAC_Encoding_EpiPerf_LowHigh/')
path_npz = join(st.path, 'figure/1_Classif_PAC_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'/')
path_rois, path_stats = join(path_npz,'Signif_ROIs'+th+'/'), join(path_npz, 'masks_stat/')
path_to_save = join(path_npz, 'masks_visbrain'+th+'/')
path_save_fig = join(path_npz, 'Signif_timecourse'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_odor_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz')
rois_form = join(path_rois, '{}_signif_min_{}_{}_min_win{}_th_{}.npy')
stat_form = join(path_stats,'{}_mask_stat_{}_minwin{}_th{}.npy')
mask_vis_form = join(path_to_save,'{}_mask_min{}_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
if not exists(path_save_fig):
    makedirs(path_save_fig)
############################################################################### 

freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha'] #'0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2
su_codes = {'S0':'CHAF','S1':'FERJ','S2':'LEFC','S3':'SEMC','S4':'PIRJ','S5':'MICP','S6':'VACJ'}
min_wins = ['1']#,'2','3','4','5']
methods = ['MAI_RL'] #,'aal_RL','BA','aal',
min_sigs = [1]#1,2,3,4,

for freq,min_win,min_sig,method in product(freqs,min_wins,min_sigs,methods):
    # Load the electrodes labels, electrodes significant (bool), rois significant (names)
    npz_mat = np.load(npz_form.format('All_subjects',freq))
    print(npz_mat.files)
    labels, subjects = npz_mat['s_'+method],npz_mat['su_codes']
    elecs, channels = npz_mat['s_elec'],npz_mat['s_channels']
    stats = np.load(stat_form.format('All_subjects',freq,min_win,th))
    rois = np.load(rois_form.format('s_Mai_RL',min_sig,freq[2:],min_win,th))
    #print(rois)
    mask_vis = []
    i=0
    for e in range(labels.shape[0]):
        if stats[e]==0 and labels[e] in rois:
            mask_vis.append(False)
            i=i+1
            print(subjects[e],labels[e],channels[e],elecs[e],'number of elecs',i)
            #path_file = path_classif+su_codes[subjects[e]]+'/'+freq+'/fig/'
            path_file_time = path_classif+freq+'/fig/'
            filename = su_codes[subjects[e]]+'_Pac_'+conds[0]+'_'+conds[1]+'_'+channels[e]+'_'+labels[e]+'_('+elecs[e]+').png'
            img = Image.open(path_file_time+filename)
            path_save_fig2 = path_save_fig+freq+'/'+method+'/'+labels[e]+'/'
            if not exists(path_save_fig2):
                makedirs(path_save_fig2)
            img.save(path_save_fig2+'/min_patient'+str(min_sig)+'_minwin_'+str(min_win)+'_'+filename,'png')
        else :
            mask_vis.append(True)
        #print(stats[e],labels[e], mask_vis[e])
    np.save(mask_vis_form.format(method,min_sig,freq,min_win,th),mask_vis)

-> Olfacto loaded
['su_codes', 's_labels', 's_aal', 's_elec_feat1', 's_elec', 's_xyz', 's_BA', 's_th_01', 's_aal_RL', 's_MAI_RL', 's_th_05', 's_channels', 's_da', 's_elec_feat0']
S1 Ins h4-h3 49 number of elecs 1
S1 OFC o2-o1 69 number of elecs 2
S1 Ins t2-t1 79 number of elecs 3
S3 MFG k9-k8 24 number of elecs 4
S3 MFG k12-k11 27 number of elecs 5
S4 HC b2-b1 0 number of elecs 6
S4 PHG d3-d2 26 number of elecs 7
S5 Ins h4-h3 34 number of elecs 8
['su_codes', 's_labels', 's_aal', 's_elec_feat1', 's_elec', 's_xyz', 's_BA', 's_th_01', 's_aal_RL', 's_MAI_RL', 's_th_05', 's_channels', 's_da', 's_elec_feat0']
S1 HC b2-b1 5 number of elecs 1
S1 IFG o11-o10 78 number of elecs 2
S2 OFC o6-o5 45 number of elecs 3
S3 STG j5-j4 21 number of elecs 4
S4 MTG b'9-b'8 10 number of elecs 5
S4 OFC j3-j2 33 number of elecs 6
['su_codes', 's_labels', 's_aal', 's_elec_feat1', 's_elec', 's_xyz', 's_BA', 's_th_01', 's_aal_RL', 's_MAI_RL', 's_th_05', 's_channels', 's_da', 's_elec_feat0']
S0 SFG c'5-c'4 3 numb